<a href="https://colab.research.google.com/github/UtkarshRaj130/Advanced_Soybean_Agricultural_Dataset_Analysis/blob/main/Code_Notebooks/07_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget https://raw.githubusercontent.com/UtkarshRaj130/Advanced_Soybean_Agricultural_Dataset_Analysis/refs/heads/main/Dataset/2_clusters_only_on_output.csv

--2025-04-13 10:08:02--  https://raw.githubusercontent.com/UtkarshRaj130/Advanced_Soybean_Agricultural_Dataset_Analysis/refs/heads/main/Dataset/2_clusters_only_on_output.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5557 (5.4K) [text/plain]
Saving to: ‘2_clusters_only_on_output.csv’

2_clusters_only_on_ 100%[===================>]   5.43K  --.-KB/s    in 0s      

2025-04-13 10:08:02 (44.6 MB/s) - ‘2_clusters_only_on_output.csv’ saved [5557/5557]



In [36]:
!wget https://raw.githubusercontent.com/UtkarshRaj130/Advanced_Soybean_Agricultural_Dataset_Analysis/refs/heads/main/Dataset/3_one_hot_encoded.csv

--2025-04-13 10:36:45--  https://raw.githubusercontent.com/UtkarshRaj130/Advanced_Soybean_Agricultural_Dataset_Analysis/refs/heads/main/Dataset/3_one_hot_encoded.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13442 (13K) [text/plain]
Saving to: ‘3_one_hot_encoded.csv’

3_one_hot_encoded.c 100%[===================>]  13.13K  --.-KB/s    in 0.003s  

2025-04-13 10:36:46 (4.89 MB/s) - ‘3_one_hot_encoded.csv’ saved [13442/13442]



In [1]:
import numpy as np
import pandas as pd
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.feature_selection import mutual_info_regression

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score, adjusted_rand_score
from sklearn.model_selection import ParameterGrid

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.model_selection import train_test_split

# Data
This dataset contains the classes obtained upon binary clustering of selected parameters from previous steps

In [3]:
data=pd.read_csv('2_clusters_only_on_output.csv')

In [4]:
data

,Sugars (Su),Weight of 300 Seeds (W3S),Seed Yield per Unit Area (SYUA),Number of Seeds per Pod (NSP),Protein Percentage (PPE),Protein Content (PCO),Cluster,Cluster Label
0,0.433,33.6,5567.4,1.86,33.2,0.82,1,Poor Production
1,0.334,34.4,2245.5,1.87,38.5,0.13,1,Poor Production
2,0.490,35.2,4326.7,2.04,33.6,0.15,1,Poor Production
3,0.163,52.3,6214.5,2.20,33.5,0.14,0,Good Production
4,0.795,31.2,3897.8,1.88,39.6,0.60,1,Poor Production
...,...,...,...,...,...,...,...,...
103,0.663,31.2,4472.7,2.60,39.7,0.32,0,Good Production
104,0.165,34.5,5643.7,2.60,33.6,0.36,0,Good Production
105,1.115,37.2,3067.1,1.80,37.2,1.15,1,Poor Production
106,0.295,35.3,3112.2,2.10,32.3,0.17,1,Poor Production


In [37]:
data_original=pd.read_csv('3_one_hot_encoded.csv')

In [38]:
data_original

,Parameters,Random,Plant Height (PH),Number of Pods (NP),Biological Weight (BW),Sugars (Su),Relative Water Content in Leaves (RWCL),ChlorophyllA663,Chlorophyllb649,Protein Percentage (PPE),...,Number of Seeds per Pod (NSP),Protein Content (PCO),C_2,S_2,S_3,G_2,G_3,G_4,G_5,G_6
0,C1S1G5,R1,50.5,130.3,111,0.433,0.732,1.4,3.1,33.2,...,1.86,0.82,False,False,False,False,False,False,True,False
1,C2S3G4,R1,44.5,132.0,80,0.334,0.674,7.1,2.2,38.5,...,1.87,0.13,True,False,True,False,False,True,False,False
2,C2S1G6,R1,52.2,150.0,83,0.490,0.677,1.8,1.3,33.6,...,2.04,0.15,True,False,False,False,False,False,False,True
3,C1S1G1,R1,50.5,140.8,66,0.163,0.745,1.1,3.0,33.5,...,2.20,0.14,False,False,False,False,False,False,False,False
4,C1S2G6,R2,49.2,175.6,73,0.795,0.725,7.5,2.1,39.6,...,1.88,0.60,False,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,C1S2G3,R3,44.0,209.0,132,0.663,0.875,9.0,1.3,39.7,...,2.60,0.32,False,True,False,False,True,False,False,False
104,C1S3G2,R2,53.3,160.0,130,0.165,0.651,1.8,3.2,33.6,...,2.60,0.36,False,False,True,True,False,False,False,False
105,C2S1G5,R2,50.2,148.3,151,1.115,0.683,6.1,1.2,37.2,...,1.80,1.15,True,False,False,False,False,False,True,False
106,C2S2G4,R1,52.2,156.0,66,0.295,0.632,6.9,1.1,32.3,...,2.10,0.17,True,True,False,False,False,True,False,False


In [63]:
def training_variable_split(data):
  training_variables=data.columns[-8:]
  data_X=pd.DataFrame(data[training_variables])
  for i in training_variables:
    data_X[i]=data_X[i].map({False:0,True:1})
  # data_X=data_X.map({'False':0,'True':1})
  return data_X

In [64]:
def target_variable_split(data):
  target=data.columns[-2]
  data_Y=pd.DataFrame(data[target])
  return data_Y

In [65]:
data_Y=target_variable_split(data)

In [66]:
data_X=training_variable_split(data_original)

In [67]:
data_X

,C_2,S_2,S_3,G_2,G_3,G_4,G_5,G_6
0,0,0,0,0,0,0,1,0
1,1,0,1,0,0,1,0,0
2,1,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...
103,0,1,0,0,1,0,0,0
104,0,0,1,1,0,0,0,0
105,1,0,0,0,0,0,1,0
106,1,1,0,0,0,1,0,0


In [68]:
def train_test_split_data(data_X,data_Y):
  data_X_train,data_X_temp,data_Y_train,data_Y_temp=train_test_split(data_X,data_Y,test_size=0.4,random_state=42)
  data_X_valid,data_X_test,data_Y_valid,data_Y_test=train_test_split(data_X_temp,data_Y_temp,test_size=0.5,random_state=42)
  return data_X_train,data_X_valid,data_X_test,data_Y_train,data_Y_valid,data_Y_test

In [69]:
data_X_train,data_X_valid,data_X_test,data_Y_train,data_Y_valid,data_Y_test=train_test_split_data(data_X,data_Y)

In [70]:
data.drop(columns=['Cluster Label']).corr()

,Sugars (Su),Weight of 300 Seeds (W3S),Seed Yield per Unit Area (SYUA),Number of Seeds per Pod (NSP),Protein Percentage (PPE),Protein Content (PCO),Cluster
Sugars (Su),1.000000,-0.250074,-0.255296,-0.205621,-0.000679,0.450942,0.332363
Weight of 300 Seeds (W3S),-0.250074,1.000000,0.438026,0.057696,-0.035110,0.072685,-0.529496
Seed Yield per Unit Area (SYUA),-0.255296,0.438026,1.000000,0.353017,0.032834,-0.023012,-0.640753
Number of Seeds per Pod (NSP),-0.205621,0.057696,0.353017,1.000000,0.092262,-0.263086,-0.609895
Protein Percentage (PPE),-0.000679,-0.035110,0.032834,0.092262,1.000000,-0.036795,-0.130952
Protein Content (PCO),0.450942,0.072685,-0.023012,-0.263086,-0.036795,1.000000,0.081490
Cluster,0.332363,-0.529496,-0.640753,-0.609895,-0.130952,0.081490,1.000000


# **Random Forest Algorithm**

In [77]:
hyper_parameters={
    'n_estimators':[20,50,100],
    'criterion':['gini','entropy'],
    'max_depth':[3,5,7],
    'min_samples_split':[2,5,10],
    'min_samples_leaf':[1,2,4]
}
randomForest=RandomForestClassifier()

In [72]:
def GridSearch_BestModel(model,hyper_parameters):
  grid_search=GridSearchCV(model,hyper_parameters,cv=5,scoring='accuracy',n_jobs=-1)
  grid_search.fit(data_X_train,data_Y_train)
  best_model=grid_search.best_estimator_
  print(grid_search.best_params_)
  return best_model

In [73]:
best_randomForest=GridSearch_BestModel(randomForest,hyper_parameters)

{'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 20}


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


{'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 20}

In [75]:
def evaluate_model(model,data_X_valid,data_Y_valid,data_X_test,data_Y_test):
  evaluation={'Accuracy':[],'Precision':[],'Recall':[],'F1 Score':[]}
  predicted_valid=model.predict(data_X_valid)
  predicted_test=model.predict(data_X_test)

  print('Accuracy Score')
  print('Validation Set:',accuracy_score(data_Y_valid,predicted_valid))
  print('Test Set:',accuracy_score(data_Y_test,predicted_test))

  print('\nPrecision Score')
  print('Validation Set: ',precision_score(data_Y_valid,predicted_valid))
  print('Test Set: ',precision_score(data_Y_test,predicted_test))

  print('\nRecall Score')
  print('Validation Set: ',recall_score(data_Y_valid,predicted_valid))
  print('Test Set: ',recall_score(data_Y_test,predicted_test))

  print('\nF1 Score')
  print('Validation Set: ',f1_score(data_Y_valid,predicted_valid))
  print('Test Set: ',f1_score(data_Y_test,predicted_test))

  evaluation['Accuracy'].append(accuracy_score(data_Y_valid,predicted_valid))
  evaluation['Accuracy'].append(accuracy_score(data_Y_test,predicted_test))
  evaluation['Precision'].append(precision_score(data_Y_valid,predicted_valid))
  evaluation['Precision'].append(precision_score(data_Y_test,predicted_test))
  evaluation['Recall'].append(recall_score(data_Y_valid,predicted_valid))
  evaluation['Recall'].append(recall_score(data_Y_test,predicted_test))
  evaluation['F1 Score'].append(f1_score(data_Y_valid,predicted_valid))
  evaluation['F1 Score'].append(f1_score(data_Y_test,predicted_test))

  print("Confusion Matrix")
  print("\nValidation Set\n")
  print(confusion_matrix(data_Y_valid,predicted_valid))
  print("\nTest Set\n")
  print(confusion_matrix(data_Y_test,predicted_test))

  return pd.DataFrame(evaluation,index=['Validation Set','Testing Set'])


In [76]:
results=evaluate_model(best_randomForest,data_X_valid,data_Y_valid,data_X_test,data_Y_test)

Accuracy Score
Validation Set: 0.7727272727272727
Test Set: 0.5909090909090909

Precision Score
Validation Set:  0.8125
Test Set:  0.6666666666666666

Recall Score
Validation Set:  0.8666666666666667
Test Set:  0.6153846153846154

F1 Score
Validation Set:  0.8387096774193549
Test Set:  0.64
Confusion Matrix
Validation Set
[[ 4  3]
 [ 2 13]]
Test Set
[[5 4]
 [5 8]]
